In [12]:
# Import Libraries
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

# Import data
decks = pd.read_csv('data.csv')
cards = pd.read_json('refs.json')

import warnings
warnings.filterwarnings("ignore")


In [13]:
# Let's only use Ranked Decks.
decks = pd.DataFrame(decks)
decks['deck_type'] = decks['deck_type'].astype('str') 
Ranked = pd.DataFrame(decks.loc[decks["deck_type"] == "Ranked Deck"])


In [14]:
sets = list(Ranked['deck_set'].value_counts(sort=False).index)

In [20]:
import matplotlib.patches as mpatches
from pylab import plot, show, savefig, xlim, figure, \
                hold, ylim, legend, boxplot, setp, axes

# Use this to find decks with any hint of popularity
Ranked2 = Ranked[Ranked["rating"] >= 5]
Ranked['year'] = Ranked['date'].astype(str).str[0:4]
Ranked2['deck_class'].apply(str) 
Ranked['year'].apply(str) 
Ranked['deck_class'].apply(str) 
#Ranked['year'].apply(str)   
# Get the card names here:
CARD_DATA = cards[['dbfId',"name","rarity","playerClass"]]
# get the name of each card:
# example:
CARD_DATA[CARD_DATA["dbfId"] == 192]["name"].value_counts(sort=False).index[0]

#years = list(Ranked['year'].value_counts(sort=False).index)
#years
# We see that we have a very uneven number of deck submissions
#print(Ranked['year'].value_counts())


'Ice Block'

In [21]:
# Use an ML Algorithm to find out something cool

# using the Ranked Dataset, we'll try to classify the upvote levels by card-counts
# See if we can classify the set by the card numbers 

# First try to classify the decks by set number
Ranked_MLA = Ranked

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report, f1_score, cohen_kappa_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

def NB_Classify(Ranked, log = True):
    card_complete = ['card_{}'.format(str(i)) for i in range(30)]
    X = Ranked[card_complete][Ranked['year']!= "2013"].applymap(str)
    Y = Ranked['year'][Ranked['year']!= "2013"]
    # train / test split
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)
    # Naive Bayes Classifier
    clf = MultinomialNB(alpha=0.1)
    clf.fit(X_train, Y_train)
    pred = clf.predict(X_test)
    
    # 
    clf2 = DummyClassifier()
    clf2.fit(X_train, Y_train)
    pred2 = clf2.predict(X_test)
    
    # metrics
    Dummy_conf_matrix = pd.DataFrame(
        confusion_matrix(Y_test, pred2), 
        columns=clf.classes_, 
        index=clf.classes_
    )
    
    conf_matrix = pd.DataFrame(
        confusion_matrix(Y_test, pred), 
        columns=clf.classes_, 
        index=clf.classes_
    )
    
    if log:
        print("Random Predictions")
        print (classification_report(Y_test, pred2))
        print (cohen_kappa_score(Y_test, pred2))
        print (f1_score(Y_test, pred2, average = "weighted"))
        print (Dummy_conf_matrix)
        print("\n")
        
        print("Naive Bayes Predictions")
        print (classification_report(Y_test, pred))
        print (cohen_kappa_score(Y_test, pred))
        print (f1_score(Y_test, pred, average = "weighted"))
        print (conf_matrix)
    
    return clf

NB_Classify(Ranked, log = True)

Random Predictions
             precision    recall  f1-score   support

       2014       0.13      0.13      0.13      5185
       2015       0.30      0.30      0.30     12282
       2016       0.48      0.48      0.48     19583
       2017       0.08      0.08      0.08      3423

avg / total       0.35      0.35      0.35     40473

-0.003457794667190983
0.3473784100813215
      2014  2015  2016  2017
2014   658  1586  2505   436
2015  1642  3649  5898  1093
2016  2499  5992  9480  1612
2017   461  1002  1693   267


Naive Bayes Predictions
             precision    recall  f1-score   support

       2014       0.16      0.15      0.16      5185
       2015       0.53      0.79      0.63     12282
       2016       0.83      0.40      0.54     19583
       2017       0.21      0.48      0.29      3423

avg / total       0.60      0.49      0.50     40473

0.2938896287410311
0.4978979037119365
      2014  2015  2016  2017
2014   789  4375    11    10
2015  2254  9742   172   114
20

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)